# Cardinality-Constrained Portfolio Optimization

## Problem Overview

Standard Markowitz portfolio optimization is a Quadratic Programming problem. However, adding **cardinality constraints** makes it NP-Hard.

### Constraints:
1. **Cardinality (K)**: Hold exactly K stocks out of N available
2. **Floor (ε)**: If you buy a stock, hold at least ε% of portfolio
3. **Ceiling (δ)**: If you buy a stock, hold at most δ% of portfolio

### Mathematical Formulation:

**Minimize:** $\sigma^2_p = \sum_{i=1}^{N} \sum_{j=1}^{N} w_i w_j \sigma_{ij}$ (portfolio variance)

**Subject to:**
- $\sum_{i=1}^{N} w_i \mu_i = R$ (target return)
- $\sum_{i=1}^{N} w_i = 1$ (weights sum to 1)
- $\sum_{i=1}^{N} z_i = K$ (exactly K assets)
- $\epsilon \cdot z_i \leq w_i \leq \delta \cdot z_i$ (floor/ceiling constraints)
- $z_i \in \{0, 1\}$ (binary selection variable)

We'll use a **Genetic Algorithm** to solve this NP-Hard problem.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

## Step 1: Download S&P 500 Stock Data

In [2]:
def get_sp500_tickers():
    tickers = [
        'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'INTC', 'AMD', 'NFLX',
        'ADBE', 'CRM', 'CSCO', 'AVGO', 'QCOM', 'TXN', 'INTU', 'AMAT', 'ASML', 'LRCX',
        'JNJ', 'UNH', 'PFE', 'ABBV', 'TMO', 'MRK', 'LLY', 'AMGN', 'GILD', 'VRTX',
        'V', 'MA', 'JPM', 'BAC', 'WFC', 'GS', 'MS', 'BLK', 'SCHW', 'CME',
        'WMT', 'PG', 'KO', 'PEP', 'MCD', 'NKE', 'SBUX', 'HD', 'LOW', 'TJX',
        'BA', 'CAT', 'GE', 'MMM', 'HON', 'ITW', 'LMT', 'RTX', 'NOC', 'GD',
        'XOM', 'CVX', 'COP', 'MPC', 'PSX', 'VLO', 'EOG', 'SLB', 'HAL', 'OKE'
    ]
    return tickers

sp500_tickers = get_sp500_tickers()
print(f'Total stocks: {len(sp500_tickers)}')
print(f'First 10 tickers: {sp500_tickers[:10]}')

Total stocks: 70
First 10 tickers: ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'INTC', 'AMD', 'NFLX']


In [3]:
def download_stock_data(tickers, start_date, end_date, max_stocks=100):
    print(f'Downloading data for up to {max_stocks} stocks...')
    tickers_subset = tickers[:max_stocks]
    data = yf.download(tickers_subset, start=start_date, end=end_date, progress=True, auto_adjust=True)['Close']
    data = data.dropna(axis=1, how='any')
    print(f'Successfully downloaded data for {len(data.columns)} stocks')
    return data

end_date = datetime.now()
start_date = end_date - timedelta(days=730)
price_data = download_stock_data(sp500_tickers, start_date, end_date, max_stocks=80)

[*********************100%***********************]  70 of 70 completed

Successfully downloaded data for 70 stocks


In [4]:
returns = price_data.pct_change().dropna()
expected_returns = returns.mean() * 252
cov_matrix = returns.cov() * 252
stock_names = returns.columns.tolist()
n_stocks = len(stock_names)

print(f'Number of stocks: {n_stocks}')
print(f'\nExpected Annual Returns (sample):')
print(expected_returns.head(10))

Number of stocks: 70

Expected Annual Returns (sample):
Ticker
AAPL    0.237558
ABBV    0.301292
ADBE   -0.249358
AMAT    0.413675
AMD     0.454292
AMGN    0.188817
AMZN    0.280499
ASML    0.349712
AVGO    0.870827
BA     -0.008016
dtype: float64


## Step 2: Define Portfolio Problem & Genetic Algorithm

In [5]:
class CardinalityConstrainedPortfolio:
    def __init__(self, expected_returns, cov_matrix, K=10, epsilon=0.01, delta=0.30):
        self.mu = expected_returns.values
        self.sigma = cov_matrix.values
        self.n = len(expected_returns)
        self.K = K
        self.epsilon = epsilon
        self.delta = delta
        self.stock_names = expected_returns.index.tolist()
    
    def portfolio_return(self, weights):
        return np.dot(weights, self.mu)
    
    def portfolio_variance(self, weights):
        return np.dot(weights, np.dot(self.sigma, weights))
    
    def portfolio_std(self, weights):
        return np.sqrt(self.portfolio_variance(weights))

In [6]:
class GeneticAlgorithm:
    def __init__(self, portfolio, pop_size=100, generations=200, crossover_rate=0.8, 
                 mutation_rate=0.1, elite_size=5, target_return=None, risk_free_rate=0.02):
        self.portfolio = portfolio
        self.pop_size = pop_size
        self.generations = generations
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.elite_size = elite_size
        self.target_return = target_return
        self.risk_free_rate = risk_free_rate
        self.n = portfolio.n
        self.K = portfolio.K
        self.epsilon = portfolio.epsilon
        self.delta = portfolio.delta
    
    def initialize_population(self):
        population = []
        for _ in range(self.pop_size):
            selected = np.random.choice(self.n, self.K, replace=False)
            weights = np.zeros(self.n)
            raw_weights = np.random.uniform(self.epsilon, self.delta, self.K)
            weights[selected] = self._normalize_weights(raw_weights)
            population.append(weights)
        return np.array(population)
    
    def _normalize_weights(self, weights):
        weights = np.clip(weights, self.epsilon, self.delta)
        total = np.sum(weights)
        if total > 0:
            weights = weights / total
        for _ in range(10):
            weights = np.clip(weights, self.epsilon, self.delta)
            total = np.sum(weights)
            if abs(total - 1.0) < 1e-6:
                break
            weights = weights / total
        return weights
    
    def fitness(self, weights):
        port_return = self.portfolio.portfolio_return(weights)
        port_std = self.portfolio.portfolio_std(weights)
        if self.target_return is not None:
            return_penalty = 100 * abs(port_return - self.target_return)
            return -port_std - return_penalty
        else:
            if port_std > 0:
                return (port_return - self.risk_free_rate) / port_std
            return -np.inf
    
    def tournament_selection(self, population, fitness_scores, tournament_size=3):
        indices = np.random.choice(len(population), tournament_size, replace=False)
        best_idx = indices[np.argmax(fitness_scores[indices])]
        return population[best_idx].copy()
    
    def crossover(self, parent1, parent2):
        if np.random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()
        selected1 = np.where(parent1 > 1e-6)[0]
        selected2 = np.where(parent2 > 1e-6)[0]
        all_selected = np.unique(np.concatenate([selected1, selected2]))
        if len(all_selected) >= self.K:
            new_selected = np.random.choice(all_selected, self.K, replace=False)
        else:
            remaining = np.setdiff1d(np.arange(self.n), all_selected)
            extra = np.random.choice(remaining, self.K - len(all_selected), replace=False)
            new_selected = np.concatenate([all_selected, extra])
        child1 = np.zeros(self.n)
        child2 = np.zeros(self.n)
        for idx in new_selected:
            w1 = parent1[idx] if parent1[idx] > 0 else np.random.uniform(self.epsilon, self.delta)
            w2 = parent2[idx] if parent2[idx] > 0 else np.random.uniform(self.epsilon, self.delta)
            alpha = np.random.random()
            child1[idx] = alpha * w1 + (1 - alpha) * w2
            child2[idx] = (1 - alpha) * w1 + alpha * w2
        child1[new_selected] = self._normalize_weights(child1[new_selected])
        child2[new_selected] = self._normalize_weights(child2[new_selected])
        return child1, child2
    
    def mutate(self, individual):
        if np.random.random() > self.mutation_rate:
            return individual
        mutated = individual.copy()
        selected = np.where(mutated > 1e-6)[0]
        if np.random.random() < 0.5 and len(selected) == self.K:
            remove_idx = np.random.choice(selected)
            unselected = np.where(mutated <= 1e-6)[0]
            if len(unselected) > 0:
                add_idx = np.random.choice(unselected)
                mutated[add_idx] = mutated[remove_idx]
                mutated[remove_idx] = 0
                selected = np.where(mutated > 1e-6)[0]
        else:
            for idx in selected:
                if np.random.random() < 0.3:
                    mutated[idx] *= np.random.uniform(0.8, 1.2)
        if len(selected) > 0:
            mutated[selected] = self._normalize_weights(mutated[selected])
        return mutated
    
    def repair(self, individual):
        selected = np.where(individual > 1e-6)[0]
        if len(selected) > self.K:
            sorted_idx = selected[np.argsort(individual[selected])[::-1]]
            for idx in sorted_idx[self.K:]:
                individual[idx] = 0
        elif len(selected) < self.K:
            unselected = np.where(individual <= 1e-6)[0]
            n_add = self.K - len(selected)
            if len(unselected) >= n_add:
                add_idx = np.random.choice(unselected, n_add, replace=False)
                for idx in add_idx:
                    individual[idx] = np.random.uniform(self.epsilon, self.delta)
        selected = np.where(individual > 1e-6)[0]
        if len(selected) > 0:
            individual[selected] = self._normalize_weights(individual[selected])
        return individual
    
    def evolve(self, verbose=True):
        population = self.initialize_population()
        best_solution = None
        best_fitness = -np.inf
        history = []
        for gen in range(self.generations):
            fitness_scores = np.array([self.fitness(ind) for ind in population])
            gen_best_idx = np.argmax(fitness_scores)
            if fitness_scores[gen_best_idx] > best_fitness:
                best_fitness = fitness_scores[gen_best_idx]
                best_solution = population[gen_best_idx].copy()
            history.append(best_fitness)
            if verbose and gen % 20 == 0:
                port_ret = self.portfolio.portfolio_return(best_solution)
                port_std = self.portfolio.portfolio_std(best_solution)
                print(f'Gen {gen}: Fitness={best_fitness:.4f}, Return={port_ret:.2%}, Risk={port_std:.2%}')
            elite_idx = np.argsort(fitness_scores)[-self.elite_size:]
            new_population = [population[i].copy() for i in elite_idx]
            while len(new_population) < self.pop_size:
                parent1 = self.tournament_selection(population, fitness_scores)
                parent2 = self.tournament_selection(population, fitness_scores)
                child1, child2 = self.crossover(parent1, parent2)
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)
                child1 = self.repair(child1)
                child2 = self.repair(child2)
                new_population.extend([child1, child2])
            population = np.array(new_population[:self.pop_size])
        return best_solution, best_fitness, history

## Step 3: Construct Efficient Frontier

In [7]:
K = 10
epsilon = 0.02
delta = 0.30
portfolio = CardinalityConstrainedPortfolio(expected_returns, cov_matrix, K=K, epsilon=epsilon, delta=delta)
print(f'Portfolio Constraints: K={K}, ε={epsilon:.0%}, δ={delta:.0%}, Available stocks={portfolio.n}')

Portfolio Constraints: K=10, ε=2%, δ=30%, Available stocks=70


In [8]:
def construct_efficient_frontier(portfolio, n_points=10, generations=80):
    min_return = portfolio.mu.min()
    max_return = portfolio.mu.max()
    target_returns = np.linspace(max(min_return, 0.0), min(max_return, 0.5), n_points)
    frontier_returns, frontier_risks, frontier_portfolios = [], [], []
    print('Constructing Efficient Frontier...')
    for i, target_ret in enumerate(target_returns):
        print(f'Point {i+1}/{n_points}: Target Return = {target_ret:.2%}')
        ga = GeneticAlgorithm(portfolio, pop_size=80, generations=generations, target_return=target_ret, crossover_rate=0.85, mutation_rate=0.15)
        best_weights, _, _ = ga.evolve(verbose=False)
        actual_return = portfolio.portfolio_return(best_weights)
        actual_risk = portfolio.portfolio_std(best_weights)
        frontier_returns.append(actual_return)
        frontier_risks.append(actual_risk)
        frontier_portfolios.append(best_weights)
        print(f'  Achieved: Return={actual_return:.2%}, Risk={actual_risk:.2%}')
    return frontier_returns, frontier_risks, frontier_portfolios

frontier_returns, frontier_risks, frontier_portfolios = construct_efficient_frontier(portfolio, n_points=10, generations=80)

Constructing Efficient Frontier...
Point 1/10: Target Return = 0.00%
  Achieved: Return=-0.00%, Risk=14.14%
Point 2/10: Target Return = 5.56%
  Achieved: Return=5.56%, Risk=14.47%
Point 3/10: Target Return = 11.11%
  Achieved: Return=11.11%, Risk=12.87%
Point 4/10: Target Return = 16.67%
  Achieved: Return=16.67%, Risk=14.29%
Point 5/10: Target Return = 22.22%
  Achieved: Return=22.22%, Risk=16.30%
Point 6/10: Target Return = 27.78%
  Achieved: Return=27.78%, Risk=16.87%
Point 7/10: Target Return = 33.33%
  Achieved: Return=33.33%, Risk=14.31%
Point 8/10: Target Return = 38.89%
  Achieved: Return=38.89%, Risk=17.99%
Point 9/10: Target Return = 44.44%
  Achieved: Return=44.44%, Risk=20.16%
Point 10/10: Target Return = 50.00%
  Achieved: Return=50.00%, Risk=22.16%


## Step 4: Interactive Efficient Frontier (Plotly)

In [9]:
# Create interactive efficient frontier plot
fig = go.Figure()

# Add individual stocks
stock_risks = np.sqrt(np.diag(cov_matrix.values))
stock_returns = expected_returns.values
fig.add_trace(go.Scatter(
    x=stock_risks,
    y=stock_returns,
    mode='markers',
    name='Individual Stocks',
    marker=dict(size=6, color='red', opacity=0.6),
    text=stock_names,
    hovertemplate='<b>%{text}</b><br>Risk: %{x:.2%}<br>Return: %{y:.2%}<extra></extra>'
))

# Add efficient frontier
fig.add_trace(go.Scatter(
    x=frontier_risks,
    y=frontier_returns,
    mode='lines+markers',
    name='Efficient Frontier (GA)',
    line=dict(color='blue', width=3),
    marker=dict(size=10, color='green'),
    hovertemplate='Risk: %{x:.2%}<br>Return: %{y:.2%}<extra></extra>'
))

# Find and highlight optimal portfolio (max Sharpe ratio)
risk_free_rate = 0.02
sharpe_ratios = [(r - risk_free_rate) / s if s > 0 else 0 for r, s in zip(frontier_returns, frontier_risks)]
best_idx = np.argmax(sharpe_ratios)
best_risk = frontier_risks[best_idx]
best_return = frontier_returns[best_idx]
best_sharpe = sharpe_ratios[best_idx]

fig.add_trace(go.Scatter(
    x=[best_risk],
    y=[best_return],
    mode='markers',
    name='Optimal Portfolio (Max Sharpe)',
    marker=dict(size=20, color='yellow', symbol='star'),
    hovertemplate=f'<b>Optimal Portfolio</b><br>Risk: %{{x:.2%}}<br>Return: %{{y:.2%}}<br>Sharpe: {best_sharpe:.3f}<extra></extra>'
))

# Update layout
fig.update_layout(
    title=f'Cardinality-Constrained Efficient Frontier<br><sub>K={K} stocks, ε={epsilon:.0%}, δ={delta:.0%}</sub>',
    xaxis_title='Risk (Standard Deviation)',
    yaxis_title='Expected Return',
    hovermode='closest',
    width=1000,
    height=700,
    template='plotly_white',
    xaxis=dict(tickformat='.0%'),
    yaxis=dict(tickformat='.0%'),
    font=dict(size=12)
)

fig.show()

## Step 5: Analyze Optimal Portfolio

In [10]:
best_weights = frontier_portfolios[best_idx]
print('='*60)
print('OPTIMAL PORTFOLIO (Maximum Sharpe Ratio)')
print('='*60)
print(f'Expected Return: {best_return:.2%}')
print(f'Risk (Std Dev):  {best_risk:.2%}')
print(f'Sharpe Ratio:    {best_sharpe:.3f}')
print(f'\nPortfolio Composition:')
print('-'*60)
selected_idx = np.where(best_weights > 1e-6)[0]
for idx in selected_idx:
    stock = stock_names[idx]
    weight = best_weights[idx]
    stock_ret = expected_returns.iloc[idx]
    print(f'  {stock:6s}: {weight:6.2%} (Annual Return: {stock_ret:7.2%})')
print('-'*60)
print(f'Total Weight: {np.sum(best_weights):.4f}')
print(f'Number of Stocks: {len(selected_idx)}')

OPTIMAL PORTFOLIO (Maximum Sharpe Ratio)
Expected Return: 33.33%
Risk (Std Dev):  14.31%
Sharpe Ratio:    2.189

Portfolio Composition:
------------------------------------------------------------
  ABBV  :  6.37% (Annual Return:  30.13%)
  AMAT  : 10.07% (Annual Return:  41.37%)
  AVGO  : 12.58% (Annual Return:  87.08%)
  GD    :  8.58% (Annual Return:  18.97%)
  GILD  : 10.84% (Annual Return:  30.09%)
  JNJ   : 11.73% (Annual Return:  17.62%)
  MA    :  6.98% (Annual Return:  17.77%)
  PG    :  6.98% (Annual Return:   3.97%)
  WMT   : 14.35% (Annual Return:  43.35%)
  XOM   : 11.52% (Annual Return:  13.85%)
------------------------------------------------------------
Total Weight: 1.0000
Number of Stocks: 10


## Step 6: Interactive Portfolio Composition (Plotly)

In [11]:
selected_stocks = [stock_names[i] for i in selected_idx]
selected_weights = best_weights[selected_idx]

# Create subplots: pie chart and bar chart
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type':'pie'}, {'type':'bar'}]],
    subplot_titles=('Portfolio Allocation', 'Portfolio Weights')
)

# Pie chart
fig.add_trace(
    go.Pie(
        labels=selected_stocks,
        values=selected_weights,
        hovertemplate='<b>%{label}</b><br>Weight: %{value:.2%}<extra></extra>',
        textposition='inside',
        textinfo='label+percent'
    ),
    row=1, col=1
)

# Bar chart
fig.add_trace(
    go.Bar(
        x=selected_weights,
        y=selected_stocks,
        orientation='h',
        marker=dict(color='steelblue'),
        hovertemplate='<b>%{y}</b><br>Weight: %{x:.2%}<extra></extra>',
        showlegend=False
    ),
    row=1, col=2
)

# Update layout
fig.update_xaxes(tickformat='.0%', row=1, col=2)
fig.update_layout(
    title_text=f'Optimal Portfolio Composition (Sharpe Ratio: {best_sharpe:.3f})',
    height=500,
    width=1200,
    template='plotly_white',
    font=dict(size=11)
)

fig.show()

## Step 7: GA Convergence Analysis

In [12]:
# Run GA one more time to show convergence
print('Running GA to show convergence...')
ga_demo = GeneticAlgorithm(portfolio, pop_size=80, generations=100, target_return=best_return, crossover_rate=0.85, mutation_rate=0.15)
_, _, history = ga_demo.evolve(verbose=False)

# Plot convergence
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=history,
    mode='lines',
    name='Best Fitness',
    line=dict(color='blue', width=2),
    hovertemplate='Generation: %{x}<br>Fitness: %{y:.4f}<extra></extra>'
))

fig.update_layout(
    title='Genetic Algorithm Convergence',
    xaxis_title='Generation',
    yaxis_title='Best Fitness Score',
    hovermode='x unified',
    width=1000,
    height=500,
    template='plotly_white',
    font=dict(size=12)
)

fig.show()

Running GA to show convergence...


## Summary

### What We Implemented:
1. **Downloaded S&P 500 data** using yfinance
2. **Formulated the NP-Hard problem** with cardinality constraints (K stocks, floor ε, ceiling δ)
3. **Built a Genetic Algorithm from scratch** with:
   - Population initialization respecting constraints
   - Tournament selection
   - Crossover (asset selection + weight blending)
   - Mutation (asset swap + weight adjustment)
   - Repair mechanism for constraint satisfaction
4. **Constructed the Efficient Frontier** by optimizing for different target returns
5. **Created interactive visualizations** using Plotly:
   - Efficient frontier with hover details
   - Portfolio composition (pie + bar charts)
   - GA convergence analysis

### Why GA Works Here:
- Standard QP solvers can't handle the binary cardinality constraint efficiently
- GA explores the discrete search space (which K stocks to pick)
- The repair mechanism ensures all solutions are feasible

### Connection to MATH-F212:
- **Integer Linear Programming**: Binary selection variables z_i ∈ {0,1}
- **Evolutionary Computation**: GA as a metaheuristic for NP-Hard problems

### Interactive Features:
- **Hover** over points to see exact values
- **Zoom** and **pan** to explore the frontier
- **Click** legend items to toggle visibility
- **Download** plots as PNG using the camera icon